<a href="https://colab.research.google.com/github/bird0401/image_search/blob/main/no_metric_learning%20/3_image_search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install onnxruntime-gpu==1.7.0
import onnxruntime

     |████████████████████████████████| 29.9 MB 39 kB/s 


In [6]:
import pathlib
import sys
import os
import shutil
import urllib.error
import urllib.request

import numpy as np
import onnxruntime
import PIL.Image

In [3]:
if os.path.isdir("/content/mnt") : shutil.rmtree("/content/mnt")
shutil.copytree("/content/drive/MyDrive/mnt","/content/mnt")

'/content/mnt'

# download image and exploit feature 

In [13]:
# download image file
# image of US president

def download_file(url, dst_path):
    try:
        with urllib.request.urlopen(url) as web_file:
            data = web_file.read()
            with open(dst_path, mode='wb') as local_file:
                local_file.write(data)
    except urllib.error.URLError as e:
        print(e)

url="https://static.tokyo-np.co.jp/image/article/size1/5/5/f/6/55f616b33d711265f1575b86873ea59a_1.jpg"
dst_path="/content/sample_img.jpg"
download_file(url, dst_path)

In [14]:
# exploit feature of download image

FEATURE_DIR = pathlib.Path("/content/mnt/feature/mobilenet_v3_large_100_224_feature_vector_v5")
ONNX_MODEL_PATH = pathlib.Path("/content/mnt/model/mobilenet_v3_large_100_224_feature_vector_v5.onnx")

onnx_session = onnxruntime.InferenceSession(str(ONNX_MODEL_PATH))

# input image
image_path = pathlib.Path(dst_path)

# pre-process image
image = PIL.Image.open(image_path)
image = image.convert("RGB")
image = image.resize((224, 224))
image = np.array(image, dtype=np.float32)
image = image / 255

# output feature
input_name = onnx_session.get_inputs()[0].name
output_name = onnx_session.get_outputs()[0].name 

query_feature = onnx_session.run([output_name], {input_name: np.expand_dims(image, 0)})[0][0]

# try

In [15]:
fs=np.array([[1,2],[3,4],[5,6],[7,8],[9,10],[11,12]])
q=np.array([4,3])
q_t = np.tile(q, (len(fs), 1))
q_t - fs

array([[ 3,  1],
       [ 1, -1],
       [-1, -3],
       [-3, -5],
       [-5, -7],
       [-7, -9]])

In [16]:
distances0 = np.linalg.norm(q_t - fs, ord=0, axis=1)
distances1 = np.linalg.norm(q_t - fs, ord=1, axis=1)
distances2 = np.linalg.norm(q_t - fs, ord=2, axis=1)
distances0, distances1, distances2

(array([2., 2., 2., 2., 2., 2.]),
 array([ 4.,  2.,  4.,  8., 12., 16.]),
 array([ 3.16227766,  1.41421356,  3.16227766,  5.83095189,  8.60232527,
        11.40175425]))

# calculate distances

In [17]:
# culculate distance on each image
results = []
limit = 10

for blob_index in range(21):
    object_ids_path = FEATURE_DIR / "{:04d}.object_ids.npy".format(blob_index)
    features_path = FEATURE_DIR / "{:04d}.features.npy".format(blob_index)
    
    # if not object_ids_path.exists():
    #     break

    object_ids = np.load(object_ids_path)
    features = np.load(features_path)
    assert len(object_ids) == len(features)

    query_features = np.tile(query_feature, (len(features), 1))

    distances = np.linalg.norm(query_features - features, axis=1)
    distance_indexes = np.argsort(distances)[:limit]
    results.extend(zip(object_ids[distance_indexes], distances[distance_indexes]))

for object_id, distance in sorted(results, key=lambda item: item[1])[0:limit]:
    print("{} {}".format(object_id, distance))

090827.jpg 11.56021499633789
006483.jpg 11.854053497314453
125437.jpg 12.095026969909668
024068.jpg 12.466474533081055
194436.jpg 12.488125801086426
042124.jpg 12.660187721252441
081087.jpg 12.713072776794434
011889.jpg 12.883013725280762
066905.jpg 12.889762878417969
079439.jpg 13.079289436340332
